## Execute Task A Stunt

### Setup and Connect

In [1]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np
import matplotlib.pyplot as plt

class RobotControl():
    # Initialize Function
    def __init__(self, ble):
        self.ble = ble
        self.notifiers_on = False
        self.TOF_HISTORY_LEN = 60
        self.IMU_HISTORY_LEN = 300
        self.calibration_factor = 0

        # A variable to store the latest sensor value
        self.latest_tof_front_reading = None
        
        # A list to store the history of all the sensor values
        # Each item in the list is a tuple (value, time)
        # WARNING: The list could grow really fast; you need to deal with this accordingly.
        self.tof1_readings = []
        self.imu_readings = []
        self.bot_pid_debug = []
        
        # A variable to store the latest imu reading
        self.latest_imu_reading = None
        
        # Activate notifications (if required)
        self.setup_notify()
    
    # A function to activate various notifications (if required)
    def setup_notify(self):
        ble.start_notify(ble.uuid['RX_TOF1'], self.tof1_callback_handler)
        ble.start_notify(ble.uuid['RX_IMU_PITCH'], self.imu_pitch_callback_handler)
        ble.start_notify(ble.uuid['RX_MOTOR_PID'], self.motor_pid_callback_handler)
        self.notifiers_on = True

    def stop_notify(self):
        ble.stop_notify(ble.uuid['RX_TOF1'])
        ble.stop_notify(ble.uuid['RX_IMU_PITCH'])
        ble.stop_notify(ble.uuid['RX_MOTOR_PID'])
        self.notifiers_on = False
    
    def tof1_callback_handler(self, uuid, byte_array):
        self.tof1_readings.append( self.ble.bytearray_to_float(byte_array) )
    
    def imu_pitch_callback_handler(self, uuid, byte_array):
        self.imu_readings.append( self.ble.bytearray_to_float(byte_array) )

    def motor_pid_callback_handler(self, uuid, byte_array):
        self.bot_pid_debug.append( self.ble.bytearray_to_float(byte_array) )

    def move_forward(self, speed):
        ble.send_command(CMD.MOVE_FORWARD, speed)
    
    def move_backward(self, speed):
        ble.send_command(CMD.MOVE_BACKWARD, speed)

    def tof1_callback_handler(self, uuid, byte_array):
        new_tof = self.ble.bytearray_to_float(byte_array)
        # print(f"Got TOF data: {new_tof}")
        self.tof1_readings.append(new_tof)

    def imu_pitch_callback_handler(self, uuid, byte_array):
        new_angle = self.ble.bytearray_to_float(byte_array)
        # print(f"Got angle data: {new_angle}")
        self.imu_readings.append( new_angle )
        
    def set_params(self, pid_p_rot, pid_min_power_rot, pid_p_fwd, pid_min_power_fwd, imu_hz):
        self.ble.send_command(CMD.SET_PID, f"{pid_p_rot}|{pid_min_power_rot}|{pid_p_fwd}|{pid_min_power_fwd}|{imu_hz}"); 

    def set_motor_calib(self, calib_rot, calib_fwd):
        self.ble.send_command(CMD.SET_MOTOR_CALIB, f"{calib_rot}|{calib_fwd}")
        
    def move_distance(self, dist_mm):
        self.ble.send_command(CMD.MOVE_DISTANCE, dist_mm)

    def turn_degrees(self, angle):
        self.ble.send_command(CMD.TURN_DEGREES, angle)
    
    def stop(self):
        self.ble.send_command(CMD.STOP, None)

    def do_stunt(self, dist):
        self.ble.send_command(CMD.DO_STUNT, dist)
    
    def get_data(self, expected_len):
        # ble.send_command(CMD.GET_DATA, None)
        while len(self.tof1_readings) < expected_len or len(self.bot_pid_debug) < expected_len \
            or len(self.imu_readings) < expected_len:
            print(f"Awaiting data: {len(self.tof1_readings)} / {expected_len}")
            time.sleep(1)
            ble.send_command(CMD.GET_DATA, None)

    def spin(self, left, right, time_ms):
        l = left
        r = right
        self.ble.send_command(CMD.SPIN, f"{l}|{r}|{time_ms}")
        time.sleep(time_ms / 1000)

    def turn_degrees(self, angle):
        self.ble.send_command(CMD.TURN_DEGREES, angle)
        
    def move_duration(self, power, duration, pause=False):
        self.ble.send_command(CMD.MOVE_DURATION, f"{power}|{duration}")
        time.sleep(duration / 1000)
        if pause:
            self.move_duration(0,250, pause=False)
            

    def dist_to_time(self, d):
        # distance (cm) to time (ms)
        # return round(69.04669* (d**0.644513))
        return round(60 * (d**0.644513))
    
    def move_dist_as_duration(self, dist, pause=False):
        t = self.dist_to_time(dist / 10)
        self.move_duration(75, t, pause=pause)
    
    def do_timed_stunt(self, fwd_time, fwd_power_l, fwd_power_r, back_mid_stop, back_time, back_power_l, back_power_r):
        back_stop_code = 1 if back_mid_stop else 0
        self.ble.send_command(CMD.DO_TIMED_STUNT, f"{fwd_time}|{fwd_power_l}|{fwd_power_r}|{back_stop_code}|{back_time}|{back_power_l}|{back_power_r}")

    def clear_data(self):
        self.tof1_readings = []
        self.imu_readings = []
        self.bot_pid_debug = []


### Define Robot Controller Class

In [2]:
LOG.propagate = False
ble = get_ble_controller()
ble.connect()
time.sleep(3)

2022-05-17 13:57:06,814 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:07:21:8D:B3:44
2022-05-17 13:57:10,293 | INFO     |: Connected to C0:07:21:8D:B3:44


### Run Stunt

In [3]:
bot = RobotControl(ble)

In [20]:
# def set_params(self, pid_p_rot, pid_min_power_rot, pid_p_fwd, pid_min_power_fwd, imu_hz):
bot.set_params(1.5, 80, 0.4, 35, 60)
bot.set_motor_calib(1.4, 1.3)

# Open-Loop Stunt (drift into spot)
# bot.move_duration(100,500)
# bot.spin(80,-80,900)
# bot.move_duration(100,1800)
# bot.spin(100,80,500)

# Parallel park
# bot.move_duration(0,500)
# bot.move_duration(100, 900)
# bot.stop()
# bot.turn_degrees(45)
# bot.stop()
# bot.move_duration(0,500)
# bot.spin(-80,-80,800)
# bot.stop()
# bot.move_duration(0,500)
# bot.turn_degrees(-40)
# bot.stop()

# Real stunt, like the flip thingy?

# bot.spin(150,150,1000)
# bot.spin(-200,-200,1000)
# bot.stop()

# bot.move_distance(1000)

# bot.do_timed_stunt( 900,170,0, \
#   True, \
#   1000, 255, 0)

# bot.move_distance(100)
bot.do_stunt(870)
bot.stop()

In [26]:
bot.clear_data()
bot.get_data(300)

Awaiting data: 0 / 300
Awaiting data: 0 / 300
Awaiting data: 38 / 300
Awaiting data: 73 / 300
Awaiting data: 109 / 300
Awaiting data: 146 / 300
Awaiting data: 182 / 300
Awaiting data: 219 / 300
Awaiting data: 255 / 300
Awaiting data: 292 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300
Awaiting data: 300 / 300


In [27]:
print(f"TOF: {bot.tof1_readings}")
print(f"PID: {bot.bot_pid_debug}")
print(f"IMU (times): {bot.imu_readings}")
print(len(bot.bot_pid_debug))

TOF: [1677.0, 1693.0, 1657.0, 1558.0, 1410.0, 1223.0, 1042.0, 819.0, 582.0, 361.0, 174.0, 60.0, 26.0, 95.0, 196.0, 334.0, 505.0, 714.0, 949.0, 1175.0, 1417.0, 1642.0, 1859.0, 1992.0, 2023.0, 2048.0, 2003.0, 1892.0, 1766.0, 1589.0, 1374.0, 1141.0, 903.0, 630.0, 379.0, 181.0, 55.0, 28.0, 545.0, 229.0, 456.0, 2381.0, 640.0, 259.0, 268.0, 36.0, 155.0, 2656.0, 3963.0, 4277.0, 458.0, 1964.0, 1031.0, 1825.0, 576.0, 541.0, 375.0, 462.0, 505.0, 492.0, 553.0, 872.0, 723.0, 352.0, 180.0, 699.0, 1006.0, 232.0, 95.0, 63.0, 42.0, 18.0, 0.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.

In [ ]:
bot.clear_data()